In [191]:
import pandas as pd
import numpy as np
import tweepy
from textblob import TextBlob
import seaborn as sns
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
import warnings
warnings.filterwarnings('ignore')
import re
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score,precision_score,recall_score

In [9]:
df=pd.read_csv("twitter_training.csv",header=None)

In [10]:
df.head(50)

,0,1,2,3
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
5,2401,Borderlands,Positive,im getting into borderlands and i can murder y...
6,2402,Borderlands,Positive,So I spent a few hours making something for fu...
7,2402,Borderlands,Positive,So I spent a couple of hours doing something f...
8,2402,Borderlands,Positive,So I spent a few hours doing something for fun...
9,2402,Borderlands,Positive,So I spent a few hours making something for fu...


In [15]:
df.columns = ['#', 'refers to', 'sentiment', 'text']
v_data.columns = ['#', 'refers to', 'sentiment', 'text']

In [17]:
df.head()
v_data.head()

,#,refers to,sentiment,text
0,3364,Facebook,Irrelevant,I mentioned on Facebook that I was struggling ...
1,352,Amazon,Neutral,BBC News - Amazon boss Jeff Bezos rejects clai...
2,8312,Microsoft,Negative,@Microsoft Why do I pay for WORD when it funct...
3,4371,CS-GO,Negative,"CSGO matchmaking is so full of closet hacking,..."
4,4433,Google,Neutral,Now the President is slapping Americans in the...


In [19]:
df.dropna(inplace=True, axis=0)

In [20]:
df=df.drop_duplicates()
v_data=v_data.drop_duplicates()

In [25]:
def data_processing(text):
    text = text.lower()
    text = re.sub(r"https\S+|www\S+https\S+", '',text, flags=re.MULTILINE)
    text = re.sub(r'\@w+|\#','',text)
    text = re.sub(r'[^\w\s]','',text)
    text_tokens = word_tokenize(text)
    filtered_text = [w for w in text_tokens if not w in stop_words]
    return " ".join(filtered_text)

In [30]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\win10\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [33]:
stop_words = set(stopwords.words('english'))
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\win10\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [34]:
df.text = df['text'].apply(data_processing)

In [35]:
v_data.text = v_data['text'].apply(data_processing)

In [36]:
from nltk.stem import PorterStemmer

In [38]:
stemmer=PorterStemmer()
def stemming(df):
    text=[stemmer.stem(word) for word in df]
    return df
    

In [39]:
df['text'] = df['text'].apply(lambda x: stemming(x))
v_data['text'] = v_data['text'].apply(lambda x: stemming(x))

In [40]:
count_vect=CountVectorizer(ngram_range=(1, 2)).fit(df['text'])


In [41]:
feature_names = count_vect.get_feature_names()

In [42]:
x_train =  count_vect.transform(df['text'])
y_train = df['sentiment']
x_test =  count_vect.transform(v_data['text'])
y_test = v_data['sentiment']

In [43]:
print("Size of x_train:", (x_train.shape))
print("Size of y_train:", (y_train.shape))
print("Size of x_test:", (x_test.shape))
print("Size of y_test:", (y_test.shape))

Size of x_train: (71656, 330220)
Size of y_train: (71656,)
Size of x_test: (1000, 330220)
Size of y_test: (1000,)


In [44]:
logreg = LogisticRegression()
logreg.fit(x_train, y_train)
logreg_pred = logreg.predict(x_test)
logreg_acc = accuracy_score(logreg_pred, y_test)
print("test accuracy: {:.2f}%".format(logreg_acc*100))

test accuracy: 97.80%


In [45]:
print(confusion_matrix(y_test, logreg_pred))
print("\n")
print(classification_report(y_test, logreg_pred))

[[166   3   1   2]
 [  0 261   2   3]
 [  0   1 278   6]
 [  1   1   2 273]]


              precision    recall  f1-score   support

  Irrelevant       0.99      0.97      0.98       172
    Negative       0.98      0.98      0.98       266
     Neutral       0.98      0.98      0.98       285
    Positive       0.96      0.99      0.97       277

    accuracy                           0.98      1000
   macro avg       0.98      0.98      0.98      1000
weighted avg       0.98      0.98      0.98      1000



In [48]:
SVCmodel = LinearSVC()
SVCmodel.fit(x_train, y_train)

LinearSVC()

In [49]:
svc_pred = SVCmodel.predict(x_test)
svc_acc = accuracy_score(svc_pred, y_test)
print("test accuracy: {:.2f}%".format(svc_acc*100))

test accuracy: 98.70%


In [50]:
print(confusion_matrix(y_test, svc_pred))
print("\n")
print(classification_report(y_test, svc_pred))

[[170   1   0   1]
 [  0 262   2   2]
 [  1   0 280   4]
 [  1   0   1 275]]


              precision    recall  f1-score   support

  Irrelevant       0.99      0.99      0.99       172
    Negative       1.00      0.98      0.99       266
     Neutral       0.99      0.98      0.99       285
    Positive       0.98      0.99      0.98       277

    accuracy                           0.99      1000
   macro avg       0.99      0.99      0.99      1000
weighted avg       0.99      0.99      0.99      1000



In [54]:
mnb_model = MultinomialNB()
mnb_model.fit(x_train,y_train)
y_pred = mnb_model.predict(x_test)

acc_score = accuracy_score(y_test,y_pred)
print("Accuracy Score is:",acc_score)

cnf_matrix = confusion_matrix(y_test,y_pred)
print("Confusion Matrix is:\n",cnf_matrix)

clf_report = classification_report(y_test,y_pred)
print("Classification report is \n",clf_report)

Accuracy Score is: 0.962
Confusion Matrix is:
 [[165   2   0   5]
 [  0 258   3   5]
 [  0   4 272   9]
 [  2   5   3 267]]
Classification report is 
               precision    recall  f1-score   support

  Irrelevant       0.99      0.96      0.97       172
    Negative       0.96      0.97      0.96       266
     Neutral       0.98      0.95      0.97       285
    Positive       0.93      0.96      0.95       277

    accuracy                           0.96      1000
   macro avg       0.96      0.96      0.96      1000
weighted avg       0.96      0.96      0.96      1000



In [152]:
tfidf_vect = TfidfVectorizer(lowercase=True, stop_words='english',min_df=10)
x_tfidf_vect = tfidf_vect.fit_transform(df['text'])

tf_idf_col = tfidf_vect.get_feature_names()
x_tfidf_df = pd.DataFrame(x_tfidf_vect.toarray(),columns=tf_idf_col)
x_tfidf_df

,00,000,00016,01,02,03,04,05,06,07,...,zion,zip,zoe,zombie,zombies,zone,zonestreamcx,zoom,zsmitty,zuckerberg
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71651,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
71652,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
71653,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
71654,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [196]:
tfidf_vect = TfidfVectorizer(lowercase=True, stop_words='english',min_df=10)
x_train =  tfidf_vect.fit(df['text'])
y_train = df['sentiment']
x_test =  tfidf_vect.fit(v_data['text'])
y_test = v_data['sentiment']

In [197]:
x_train =  tfidf_vect.transform(df['text'])
y_train = df['sentiment']
x_test =  tfidf_vect.transform(v_data['text'])
y_test = v_data['sentiment']

In [199]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(71656, 170)
(71656,)
(1000, 170)
(1000,)


In [171]:
rf = RandomForestClassifier()
scores = cross_val_score(rf,x_train,y_train.values.ravel(),cv=5)

In [172]:
print(scores)

[0.48032375 0.41637011 0.34359082 0.33605471 0.44449096]


In [173]:
scores.mean()

0.4041660690286645

In [162]:
rf = RandomForestClassifier()
parameters = {
    'n_estimators': [5,50,100],
    'max_depth': [2,10,20,None]
}

cv = GridSearchCV(rf,parameters)
cv.fit(x_train,y_train.values.ravel())

GridSearchCV(estimator=RandomForestClassifier(),
             param_grid={'max_depth': [2, 10, 20, None],
                         'n_estimators': [5, 50, 100]})

In [163]:
cv.best_params_

{'max_depth': None, 'n_estimators': 100}

In [188]:
rf1=RandomForestClassifier(max_depth=None,n_estimators= 100)
rf1.fit(x_train,y_train)


RandomForestClassifier()

In [189]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(71656, 170)
(71656,)
(1000, 170)
(1000,)


In [193]:
y_pred = rf1.predict(x_test)
accuracy=accuracy_score(y_test,y_pred)

In [194]:
print("testing accuracy is",accuracy)

testing accuracy is 0.774


In [195]:
print(confusion_matrix(y_test, y_pred))
print("\n")
print(classification_report(y_test, y_pred))

[[121  27   8  16]
 [  8 228  21   9]
 [  9  42 213  21]
 [ 10  41  14 212]]


              precision    recall  f1-score   support

  Irrelevant       0.82      0.70      0.76       172
    Negative       0.67      0.86      0.75       266
     Neutral       0.83      0.75      0.79       285
    Positive       0.82      0.77      0.79       277

    accuracy                           0.77      1000
   macro avg       0.79      0.77      0.77      1000
weighted avg       0.78      0.77      0.77      1000



In [11]:
v_data=pd.read_csv('twitter_validation.csv',header=None)

In [200]:
v_data.head()

,#,refers to,sentiment,text
0,3364,Facebook,Irrelevant,mentioned facebook struggling motivation go ru...
1,352,Amazon,Neutral,bbc news amazon boss jeff bezos rejects claims...
2,8312,Microsoft,Negative,microsoft pay word functions poorly samsungus ...
3,4371,CS-GO,Negative,csgo matchmaking full closet hacking truly awf...
4,4433,Google,Neutral,president slapping americans face really commi...


Here Logistic regression,support vector classifier and naive bais gave good results

Pickling down the file

In [204]:
import pickle

In [207]:
pickle.dump(logreg,open('logmodel.pkl','wb'))
pickled_model=pickle.load(open('logmodel.pkl','rb'))
